In [1]:
%%writefile laminarflow/_cruisecontrol.py
import tensorflow as tf
import pickle as pkl
    
class _tf_temp():
    def __init__(self, name):
        self.name = name
class _method_temp():
    def __init__(self, name, method_name):
        self.name = name
        self.method_name = method_name

class CruiseControl():
    """
    Laminar Flow's Cruise Control method automates several
    time saving tasks for Tensorflow, allowing for quicker
    prototyping and testing.
    
    Among these abilities is automatic saving in an encapsulated
    tensorflow session. This makes it so you don't need to
    keep open a session to keep the values. However, it does
    require you to use `tf.get_variable` to define variables
    instead of `tf.Variable` directly.
    
    On top of that, automatic initialization of uninitialized
    variables allows for this structure to be dynamically updated
    and have no cat and mouse hunt for errors.
    
    Pickling and unpickling has been implemented in this class
    under the strict conditions that all non-variable tensors
    used in args to the `add` function are the result of previous
    functions input to the `add` function and all `function`s used
    have to either be functions or methods bound to results of
    previous calls to add. That is, when calling `add`, only use
    direct attributes of that CruiseControl instance or variables
    controlled by it and only use functions or methods controlled
    by that instance as well.
    
    Since TensorFlow objects are not pickleable directly, args
    and kwargs to `add` have to be easily sanitized, unless you
    want to do it yourself. More complex sanitization is possible,
    if somewhat difficult. To sanitize yourself, currently you must
    create the magic functions in whatever you pass to `add` if
    it isn't automatically taken care of already.
    
    More documentation to follow. This all needs to be reworded
    to make more sense.
    """
#Constructor
    def __init__(self, save_file_name, unique_identifier = None):
        #collect_variables()
        self._vars = set()
        self._var_pkl = list()
        self._file_name = save_file_name
        self.saver = None
        self._uuid = unique_identifier if unique_identifier else hex(id(self))[2:]
        try:
            with tf.variable_scope(self._uuid):
                with tf.variable_scope(self._uuid):
                    tf.get_variable("initialized",shape=[1])
        except:
            raise NameError("Error: {} is an invalid unique identifier.".format(self._uuid))
#Structure
    def add(self, name, function, *args, **kwargs):
        if hasattr(self, name):
            raise AttributeError("Error: {} already defined.".format(name))
        current = set(tf.all_variables())
        #sanitize
        sanitized_args = [self.sanitize(arg) for arg in args]
        sanitized_kwargs = {key:self.sanitize(value) for key,value in kwargs.items()}
        sanitized_func = self.sanitize(function)
        #Test.
        try:
            pkl.dumps([sanitized_func, sanitized_args, sanitized_kwargs])
        except:
            raise ValueError("Error: Unable to sanitize.")
        #unsanitize
        unsanitized_func = self.unsanitize(function)
        unsanitized_args = [self.unsanitize(arg) for arg in args]
        unsanitized_kwargs = {key:self.unsanitize(value) for key,value in kwargs.items()}
        with tf.variable_scope(self._uuid):
            with tf.variable_scope(name):
                setattr(self, name, unsanitized_func(*unsanitized_args, **unsanitized_kwargs))
        self._vars |= set(tf.all_variables()) - current
        self._var_pkl.append([name, sanitized_func, sanitized_args, sanitized_kwargs])
        return self
#Sanitization
    def sanitize(self, obj):
        if hasattr(obj, "__self__"):
            method_self = getattr(obj, "__self__")
            for name,_,_,_ in self._var_pkl:
                if getattr(self, name) is method_self:
                    return _method_temp(name, obj.__name__)
            for var in self._var:
                if var is method_self:
                    start = len(self._uuid)
                    restart = var.name[start:].find('/') + start + 1
                    return _method_temp(var.name[restart:var.name.rfind(":")], obj.__name__)
        try:
            pkl.dumps(obj)
            return obj
        except:
            start = len(self._uuid)
            restart = obj.name[start:].find('/') + start + 1
            return _tf_temp(obj.name[restart:obj.name.rfind(":")])
    def unsanitize(self, obj):
        if isinstance(obj, _tf_temp):
            try:
                return tf.get_variable(obj.name)
            except:
                end = obj.name.find('/')
                return getattr(self, obj.name[:end])
        if isinstance(obj, _method_temp):
            try:
                return getattr(tf.get_variable(obj.name), obj.method_name)
            except:
                return getattr(getattr(self, obj.name), obj.method_name)
        return obj
#Features
    def setFile(self, save_file_name):
        self._file_name = save_file_name
#Values
    def save(self, save_file_name = None):
        variables = []
        start = len(self._uuid)
        for i in self._vars:
            restart = i.name[start:].find('/') + start + 1
            if tf.is_variable_initialized(i).eval():
                try:
                    variables.append((i.name[restart:i.name.rfind(":")],i.value().eval()))
                except:
                    #TODO: Don't do this. Limit exceptions to known expected ones.
                    pass
        with open(self._file_name, "wb") as file:
            pkl.dump(variables, file)
    def load(self, save_file_name = None):
        try:
            with open(self._file_name, "rb") as file:
                variables = pkl.load(file)
            with tf.variable_scope(self._uuid, reuse=True):
                for i in variables:
                    tf.get_variable(i[0]).assign(i[1]).eval(session=self._sess)
        except:
            #TODO: Don't do this. Limit exceptions to known expected ones.
            pass
#Serialization
    def __reduce__(self):
        return (CruiseControl, (self._file_name,), self._var_pkl)
    def __setstate__(self, state):
        for i in state:
            self.add(i[0],i[1],*i[2],**i[3])
        return self
    '''
#Initialization
    def initialize_variables(self, specifically=None):
        uninitialized = []
        start = len(self._uuid)
        with tf.variable_scope(self._uuid, reuse=True):
            for name in self._sess.run(tf.report_uninitialized_variables(self._vars)):
                name = name.decode("utf-8")
                restart = name[start:].find('/') + start + 1
                end = name.rfind(":")
                if end == -1:
                    end = None
                print(name)
                print(restart)
                print(end)
                print(name[restart:end])
                uninitialized.append(tf.get_variable(name[restart:end]))
        self._sess.run(tf.initialize_variables(uninitialized))
    '''
#Functionality
    def __enter__(self):
        self._sess = tf.Session()
        #self.initialize_variables()
        # trying to be smart about initialization seems
        #  to be a bad idea for some reason?
        self._sess.run(tf.initialize_all_variables())
        # We're just going to load values back anyway.
        self.load()
        return self._sess.__enter__()
    def __exit__(self, *args):
        self.save()
        return self._sess.__exit__(*args)

Overwriting laminarflow/_cruisecontrol.py


In [2]:
import tensorflow as tf

def clu(x, outdim, indim=None):
    if indim:
        try:
            indim = list(indim)
        except:
            indim = [indim]
    else:
        indim = x.get_shape()[1:].as_list()
    try:
        outdim = list(outdim)
    except:
        outdim = [outdim]
    C = tf.get_variable("C", initializer=tf.truncated_normal(indim+outdim, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(outdim, stddev=0.1))
    cluKernel = -tf.reduce_sum(tf.abs(tf.sub(tf.expand_dims(x,len(indim)+1),
                                        tf.expand_dims(C,0))),
                          1) + b
    cluKernel._C = C
    cluKernel._b = b
    return cluKernel

def lin(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(shape[1:], stddev=0.1))
    linKernel = tf.matmul(x,W) + b
    linKernel._W = W
    linKernel._b = b
    return linKernel

#Convolution and Maxpool shortcuts
def conv2d(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=shape[-1:]))
    convKernel = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') + b
    convKernel._W = W
    convKernel._b = b
    return convKernel

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def cross_entropy(y,yhat):
    return -tf.reduce_sum(y*tf.log(yhat + 1e-9)) #Without epsilon, it crashes.

def accuracy_test(y,yhat):
    correct_prediction = tf.equal(tf.argmax(yhat,1), tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [3]:
from laminarflow import CruiseControl

test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
#test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
#test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
#test.add('loss', cross_entropy, test.expected_output, test.output)
#test.add('train_step', test.optim.minimize, test.loss)

In [4]:
import numpy as np

with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))

[[  7.56635098e-04   9.22801197e-01   3.53984050e-02   5.09097939e-04
    3.00782995e-04   1.29580693e-02   6.00280566e-03   7.86879566e-04
    1.86771862e-02   1.80903543e-03]]
[[  7.56635098e-04   9.22801197e-01   3.53984050e-02   5.09097939e-04
    3.00782995e-04   1.29580693e-02   6.00280566e-03   7.86879566e-04
    1.86771862e-02   1.80903543e-03]]


In [5]:
import pickle as pkl

test2 = pkl.loads(pkl.dumps(test))
with test2 as sess:
    print(test2.output.eval({test2.input:np.ones([1,784])}))

[[  7.56635098e-04   9.22801197e-01   3.53984050e-02   5.09097939e-04
    3.00782995e-04   1.29580693e-02   6.00280566e-03   7.86879566e-04
    1.86771862e-02   1.80903543e-03]]


In [6]:
test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
test.add('loss', cross_entropy, test.expected_output, test.output)
test.add('train_step', test.optim.minimize, test.loss)
#testing
test._initialized = set(x for x in test._vars)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))

[[  7.56635098e-04   9.22801197e-01   3.53984050e-02   5.09097939e-04
    3.00782995e-04   1.29580693e-02   6.00280566e-03   7.86879566e-04
    1.86771862e-02   1.80903543e-03]]
[[ 0.01438009  0.00087475  0.01885835  0.04185215  0.06387353  0.01450145
   0.00137812  0.52542412  0.00159438  0.31726319]]


In [7]:
test._var_pkl[6][1].method_name

'minimize'

In [8]:
import pickle as pkl

test2 = pkl.loads(pkl.dumps(test))
with test2 as sess:
    print(test2.output.eval({test2.input:np.ones([1,784])}))

[[  8.37546110e-01   2.27814773e-03   9.26363617e-02   1.84260221e-04
    2.06556288e-03   1.05861656e-03   4.58063696e-05   6.26441836e-02
    1.17108016e-03   3.69892048e-04]]


In [9]:
test.add('relu', tf.nn.relu, test.h)
test.add('h2', lin, test.relu, [10,10])
test.add('output2', tf.nn.softmax, test.h2)
test.add('loss2', cross_entropy, test.expected_output, test.output2)
test.add('train_step2', test.optim.minimize, test.loss2)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output2.eval({test.input:np.ones([1,784])}))

[[ 0.58142567  0.00060284  0.00336342  0.01735914  0.03144473  0.02135924
   0.0277974   0.23094912  0.0768769   0.00882158]]
[[ 0.09610475  0.11759425  0.08281995  0.09182145  0.08793612  0.12338841
   0.10333482  0.07687112  0.08729345  0.13283578]]


In [10]:
tf.is_variable_initialized(t).eval()

NameError: name 't' is not defined

In [ ]:
import tensorflow as tf

In [ ]:
with tf.variable_scope("test2"):
    test = {}
    test['in'] = tf.placeholder("float", shape=[1,2])
    test['out'] = tf.placeholder("float", shape=[1,3])
    test['W'] = tf.get_variable("W", shape=[2,3])
    test['loss'] = tf.nn.l2_loss(tf.matmul(test['in'], test['W']) - test['out'])
    test['optim'] = tf.train.AdadeltaOptimizer()
    test['train_step'] = test['optim'].minimize(test['loss'])

In [ ]:
test['train_step'].name

In [ ]:
tf.all_variables()

In [ ]:
[i.name for i in tf.all_variables()]

In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
with tf.variable_scope("test2", reuse=True):
    W = tf.get_variable("W")
    #WAda = tf.get_variable("test2/W/Adadelta:0")
    WAda1 = tf.get_variable("test2/W/Adadelta_1:0")